Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np
import matplotlib.pyplot as plt

In [2]:
PROBLEM_SIZE = 10
NUM_SETS = 28
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [3]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def sub_goal_check(state, reduced_sets):
    return np.all(reduce(
        np.logical_or,
        [reduced_sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE//4)]),
    ))

def h(state):
    if not sampling:
        indexes = np.arange(PROBLEM_SIZE)
        indexes = indexes[np.logical_not(reduce(
                np.logical_or,
                [SETS[i] for i in state.taken],
                np.array([False for _ in range(PROBLEM_SIZE)]),
            ))]
        reduced_sets = list(np.sum(SETS[val][indexes]) for val in state.not_taken)
        reduced_sets.sort(reverse=True)
        count = 0
        k = 0
        for j in range(len(reduced_sets)):
            if count >= indexes.shape[0]:
                break
            count += reduced_sets[j]
            k += 1
        return k
    else:
        indexes = np.arange(PROBLEM_SIZE)
        indexes = indexes[np.logical_not(reduce(
                np.logical_or,
                [SETS[i] for i in state.taken],
                np.array([False for _ in range(PROBLEM_SIZE)]),
            ))]
        np.random.shuffle(indexes)
        estimates = np.zeros(4)
        for i in range(4):
            reduced_sets = list(np.sum(SETS[val][indexes[i*PROBLEM_SIZE//4:((i+1)*PROBLEM_SIZE//4)]]) for val in state.not_taken)
            reduced_sets.sort(reverse=True)
            count = 0
            k = 0
            for j in range(len(reduced_sets)):
                if count >= indexes[i*PROBLEM_SIZE//4:((i+1)*PROBLEM_SIZE//4)].shape[0]:
                    break
                count += reduced_sets[j]
                k += 1
            estimates[i] = k
        return np.max(estimates)

def g(state):
    return len(state.taken)

In [4]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [5]:
A_star = True
sampling = True

frontier = PriorityQueue()
#frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((0, state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((h(new_state) if not A_star else h(new_state)+g(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 12 steps (2 tiles)


In [6]:
current_state

State(taken={1, 20}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27})